In [1]:
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path_svm = 'Models/' + 'best_xgb.pickle'
with open(path_svm, 'rb') as data:
    svc_model = pickle.load(data)

path_tfidf = 'Pickles/' + 'tfidf.pickle'
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

category_codes = {
    'depression': 0,
    'SuicideWatch': 1
}

In [23]:

punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))


def create_features_from_text(text):
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['label'])
    df.loc[0] = text
    df['Content_Parsed_1'] = df['label'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    df = df['Content_Parsed_6']
    display(df.head())
    # df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

    # TF-IDF
    features = tfidf.transform(df).toarray()

    return features

In [24]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():
        if id_ == category_id:
            return category

In [25]:
def predict_from_text(text):
    # Predict using the input model
    prediction_svc = svc_model.predict(create_features_from_text(text))[0]
    prediction_svc_proba = svc_model.predict_proba(create_features_from_text(text))[0]

    # Return result
    category_svc = get_category_name(prediction_svc)

    print("The predicted category using the SVM model is %s." % (category_svc))
    print("The conditional probability is: %a" % (prediction_svc_proba.max() * 100))

In [29]:
predict_from_text("I am feeling suicidal today.")

C:\Users\sathy\AppData\Local\Temp\ipykernel_24812\4014501650.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
C:\Users\sathy\AppData\Local\Temp\ipykernel_24812\4014501650.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


0      feel suicidal today
Name: Content_Parsed_6, dtype: object

C:\Users\sathy\AppData\Local\Temp\ipykernel_24812\4014501650.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
C:\Users\sathy\AppData\Local\Temp\ipykernel_24812\4014501650.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


0      feel suicidal today
Name: Content_Parsed_6, dtype: object

The predicted category using the SVM model is SuicideWatch.
The conditional probability is: 85.89933514595032
